In [48]:
# Props to this sensei
# https://www.youtube.com/watch?v=kCc8FmEb1nY&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=8

In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torch.autograd import Variable
import torch.nn.functional as F


from tqdm import tqdm # progress bar

device = ("cuda" if torch.cuda.is_available() else "mps"
          if torch.backends.mps.is_available() else "cpu")
print(f"Using {device} device")

Using cpu device


## Hyper-parameters

In [73]:
text_file = "tiny-shakespeare.txt"
batch_size = 32 # how many blocks will be given to model
block_size = 12 # context length
max_iters = 6000
learning_rate = 1e-3
eval_interval = 300
eval_iters = 200
n_embd = 32
n_heads = 6
head_size = 16
dropout = 0.0

## Reading Data

In [51]:
# read file
with open(text_file, "r") as f:
    text = f.read()
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [52]:
char_list = sorted(list(set(text)))
char_size = len(char_list)
print(f"All the characters in the text: {''.join(char_list)}")
print(f"Length of the characters: {char_size}")

All the characters in the text: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Length of the characters: 65


## Tokenizer (character based, index/ascii)

In [53]:
class MyTokenizer:
    def __init__(self):
        self.char_to_index = None
        self.index_to_char = None

    def fit(self, char_list):  
        self.char_to_index = {char: idx for idx, char in enumerate(char_list)}
        self.index_to_char = {idx: char for char, idx in self.char_to_index.items()}

    def encode_index(self, input_str):
        return [self.char_to_index[char] for char in input_str]

    def decode_index(self, encoded_list):
        return ''.join([self.index_to_char[idx] for idx in encoded_list])

    @staticmethod
    def ascii_tokenizer(char):
        return ord(char)

    @staticmethod
    def ascii_decoder(ascii_value):
        return chr(ascii_value)

    def encode_combined(self, input_str, use_ascii=False):
        if use_ascii:
            return [self.ascii_tokenizer(char) for char in input_str]
        else:
            return self.encode_index(input_str)

    def decode_combined(self, encoded_list, use_ascii=False):
        if use_ascii:
            return ''.join([self.ascii_decoder(ascii_value) for ascii_value in encoded_list])
        else:
            return self.decode_index(encoded_list)

In [54]:
# Example usage:
tokenizer = MyTokenizer()
tokenizer.fit(char_list)

input_str = "Hello there"
encoded_list_ascii = tokenizer.encode_combined(input_str, use_ascii=True)
decoded_str_ascii = tokenizer.decode_combined(encoded_list_ascii, use_ascii=True)

encoded_list_index = tokenizer.encode_combined(input_str, use_ascii=True)
decoded_str_index = tokenizer.decode_combined(encoded_list_index, use_ascii=True)

print("Original String:", input_str)
print("Encoded List (ASCII):", encoded_list_ascii)
print("Decoded String (ASCII):", decoded_str_ascii)

print("Encoded List (Index):", encoded_list_index)
print("Decoded String (Index):", decoded_str_index)

Original String: Hello there
Encoded List (ASCII): [72, 101, 108, 108, 111, 32, 116, 104, 101, 114, 101]
Decoded String (ASCII): Hello there
Encoded List (Index): [72, 101, 108, 108, 111, 32, 116, 104, 101, 114, 101]
Decoded String (Index): Hello there


In [55]:
# Encode all the data and split into train and val
encoded_data = torch.tensor(tokenizer.encode_combined(text))
n = int( 0.9 * len(encoded_data))

train_data = encoded_data[:n]
val_data = encoded_data[n:]

train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52])

In [56]:
x = train_data[0:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]) the target: 64
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64]) the target: 43
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43]) the target: 52


In [57]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # tensor of random indexes (batch_size, )
    x = torch.stack([data[i:i+block_size] for i in ix]) # from the random index go until block_size, for all random indexes
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # from the random index+1 go until block_size+1, for all random indexes
    return x,y 

xb, yb = get_batch("train") # xb -> input to the transformer
print("inputs: ")
print(xb.shape)
print(xb)

print("targets: ")
print(yb.shape)
print(yb)

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs: 
torch.Size([32, 12])
tensor([[52, 47, 45, 46, 58,  8,  0, 26, 53, 61,  1, 57],
        [46,  1, 46, 47, 57,  1, 54, 39, 57, 58,  6,  0],
        [39, 47, 52,  2,  0,  0, 13, 50, 50,  1, 15, 53],
        [43, 52, 47, 58, 63,  1, 58, 53,  1, 50, 43, 41],
        [ 1, 40, 53, 58, 46,  1, 45, 53, 53, 42,  0, 39],
        [43, 56, 42, 10,  0, 32, 46, 47, 57,  1, 47, 57],
        [57, 46, 43, 57,  0, 13, 52, 42,  1, 58, 46, 43],
        [46, 39, 52, 42, 57,  1, 39, 57,  1, 53, 50, 42],
        [ 1, 52, 47, 45, 46, 58,  2,  0,  0, 28, 27, 24],
        [59, 40, 40, 43, 56, 47, 52, 45,  8,  0, 31, 58],
        [58, 12,  1, 58, 46, 39, 58,  1, 21,  1, 39, 57],
        [54, 43, 39, 49,  6,  1, 39, 52, 42,  1, 58, 53],
        [43,  1, 61, 46, 53, 57, 43,  1, 45, 56, 43, 39],
        [ 1, 49, 47, 52, 45, 57,  2,  0, 21,  1, 61, 47],
        [ 1, 58, 46, 39, 58,  1, 40, 43,  1, 56, 47, 45],
        [ 8,  0,  0, 19, 24, 27, 33, 15, 17, 31, 32, 17],
        [46, 47, 57,  1, 44, 39, 58, 46, 4

## Bigram Language Model

In [74]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

@torch.no_grad()
def estimate_loss(): # to reduce noise of the losses
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = Head(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device)) # (T,C)
        x = tok_emb+pos_emb
        x = self.sa_head(x)
        logits = self.lm_head(tok_emb)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # Logits are the outputs of a neural network before the activation function is applied.
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens): # given a idx
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:,-block_size:]
            # get the predictions
            logits, loss = self(idx_cond)  # self.forward()
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(char_size)
model.to(device)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

print(tokenizer.decode_combined(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([384, 65])
tensor(4.3745, grad_fn=<NllLossBackward0>)

OcqXg,rqvN3w!AXyailAgSjBiYYURwVUnaYq:q&OECW:;mAnSCn3oJ-uMtoaJbf$Db Gvz OuPcUhEnoJGU.cXJWpBsTLj;JYSrN


## Train Model

In [75]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 4.3277, val loss 4.3289
step 300: train loss 2.8420, val loss 2.8509
step 600: train loss 2.6366, val loss 2.6519
step 900: train loss 2.5664, val loss 2.5781
step 1200: train loss 2.5338, val loss 2.5568
step 1500: train loss 2.5021, val loss 2.5455
step 1800: train loss 2.5026, val loss 2.5255
step 2100: train loss 2.4878, val loss 2.5128
step 2400: train loss 2.4908, val loss 2.5099
step 2700: train loss 2.4831, val loss 2.5146
step 3000: train loss 2.4842, val loss 2.5160
step 3300: train loss 2.4786, val loss 2.5047
step 3600: train loss 2.4774, val loss 2.5019
step 3900: train loss 2.4771, val loss 2.4985
step 4200: train loss 2.4749, val loss 2.4953
step 4500: train loss 2.4659, val loss 2.4956
step 4800: train loss 2.4699, val loss 2.4983
step 5100: train loss 2.4694, val loss 2.4933
step 5400: train loss 2.4694, val loss 2.4849
step 5700: train loss 2.4701, val loss 2.4891


In [60]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
# context = torch.tensor([tokenizer.encode_combined(list("Hello There "))], dtype=torch.long, device=device)
print(tokenizer.decode_combined(model.generate(context, max_new_tokens=100)[0].tolist()))

IndexError: index out of range in self

## Self-Attention
Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [61]:
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [62]:
# version 1: averaging past context with for loops, the weakest form of aggregation
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) # bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)
print(x[0])
print(xbow[0])


tensor([[ 0.6274,  0.3165],
        [ 2.7856, -0.7135],
        [ 0.4451,  0.9051],
        [-0.2157, -0.3202],
        [-0.3933, -0.5900],
        [ 2.0040,  0.5141],
        [ 0.7712,  0.9372],
        [ 0.1193, -1.6433]])
tensor([[ 0.6274,  0.3165],
        [ 1.7065, -0.1985],
        [ 1.2860,  0.1694],
        [ 0.9106,  0.0470],
        [ 0.6498, -0.0804],
        [ 0.8755,  0.0187],
        [ 0.8606,  0.1499],
        [ 0.7679, -0.0743]])


In [63]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T)) # Takes only the lower triangular part of of matrix, others are zero
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2) # same thing with version 3, but faster

True

In [64]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [65]:
# version 4: self-attention!
# difference from other is this creates a non-uniform weighted averages
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
key = nn.Linear(C, head_size, bias=False) # What do I contain?
query = nn.Linear(C, head_size, bias=False) # What am I looking for?
value = nn.Linear(C, head_size, bias=False) # How the token will communicate to you, 
k = key(x)   # (B, T, head_size)
q = query(x) # (B, T, head_size)
wei =  q @ k.transpose(-2, -1) # (B, T, head_size) @ (B, head_size, T) ---> (B, T, T)
wei *= head_size**-0.5 # scaled, otherwise softmax would be way too peaky!!

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # remove this to make it encoder attention block, allowing all tokens to communicate
wei = F.softmax(wei, dim=-1) 

v = value(x)
out = wei @ v
#out = wei @ x

out.shape # (B, T, head_size)

torch.Size([4, 8, 16])